# Modelo    

In [3]:
from dataclasses import dataclass
from typing import Any, List
import numpy as np

@dataclass
class Emotions:
    name : str
    quantity : int

@dataclass
class Point:
    x : float
    y : float

@dataclass
class BoundingBox:
    origin : Point
    end: Point


@dataclass
class Person:
    age : str
    emotions : List[Emotions]
    bounding_box : BoundingBox
    image : np.array

@dataclass
class FaceDetectorResult:
    image : np.array 
    bounding_box : BoundingBox

@dataclass
class FaceComparatorResult:
    similarity : float

# Aplicación

In [4]:
from abc import ABC, abstractmethod
from numpy import array
from typing import List

class FaceDetector(ABC):
    @abstractmethod
    def detect(self, image : array) -> List[FaceDetectorResult]:
        pass

class FaceQualifier(ABC):
    @abstractmethod
    def qualify(self, face_detector_result : FaceDetectorResult) -> List[Person]:
        pass

class FaceComparator(ABC):
    @abstractmethod
    def qualify(self, first_face : FaceDetectorResult, second_face: FaceDetectorResult) -> List[FaceComparatorResult]:
        pass

# Infraestructura


In [20]:
class ImageUtils:
    @staticmethod
    def crop(image : np.array, bounding_box : BoundingBox) -> np.array:
        return image[bounding_box.origin.x:bounding_box.end.x, bounding_box.origin.y:bounding_box.end.y]

In [27]:
import os
import cv2
from ultralytics import YOLO
from retinaface import RetinaFace
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

class ViolaJonesFaceDetector(FaceDetector):
    cascPathface = os.path.dirname(
        cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"


    def detect(self, image : array) -> List[FaceDetectorResult]:
        faceCascade = cv2.CascadeClassifier(self.cascPathface)
        gray = self._convert_image_to_gray(image)
        faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
        return self._convert_to_face_detector_result(image, faces)
        
    
    def _convert_to_face_detector_result(self, image, faces : List[array]) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(x+w, y+h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]
    
    def _convert_image_to_gray(self, image : array) -> array:
        import cv2
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

class RetinafaceFaceDetector(FaceDetector):
    def detect(self, image : array) -> List[FaceDetectorResult]:
        faces = RetinaFace.detect_faces(image);
        return self._convert_to_face_detector_result(image, [faces[key]['facial_area'] for key in faces.keys()])
    
    def _convert_to_face_detector_result(self, image, faces : List[array]) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(w, h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]

class MediaPipeFaceDetector(FaceDetector):
    def detect(self, image : array) -> List[FaceDetectorResult]:
        BaseOptions = mp.tasks.BaseOptions
        FaceDetector = mp.tasks.vision.FaceDetector
        FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
        VisionRunningMode = mp.tasks.vision.RunningMode

        options = FaceDetectorOptions(
            base_options=BaseOptions(model_asset_path='./blaze_face_short_range.tflite'),
            running_mode=VisionRunningMode.IMAGE)

        with FaceDetector.create_from_options(options) as detector:
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            face_detector_result = detector.detect(mp_image)
            return self._convert_to_face_detector_result(image, [(x.bounding_box.origin_x, x.bounding_box.origin_y, x.bounding_box.width, x.bounding_box.height) for x in face_detector_result.detections])
        
    def _convert_to_face_detector_result(self, image, faces : List[array]) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(x + w, y + h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]
    

In [30]:
class DeepfaceFaceQualifier(FaceQualifier):
    def qualify(self, face_detector_result : FaceDetectorResult) -> List[Person]:
        from deepface import DeepFace
        import pandas as pd
        import numpy as np
        import cv2
        import os
        
        return [Person("42", [Emotions("happy", 0.42)], face_detector_result.bounding_box, face_detector_result.image)]

# CV2 INTERFACE

In [ ]:
class FaceQualificationDisplay(ABC):
    @abstractmethod
    def display(self) -> None:
        pass

class OpenCVFaceQualificationDisplay(ABC):
    def __init__(self, people : List[Person], frame : np.array) -> None:
        self.people = people
    
    def display(self):
        pass

In [31]:
import cv2 as cv

video = cv.VideoCapture(0)

while True:
    ret, frame = video.read()
    face_detector = MediaPipeFaceDetector()
    faces = face_detector.detect(frame)
    for x in faces:
        cv.rectangle(frame, (x.bounding_box.origin.x, x.bounding_box.origin.y), (x.bounding_box.end.x, x.bounding_box.end.y), (255, 0, 0), 2)
    
    for x in faces:
        qualifier = DeepfaceFaceQualifier()
        qualification = qualifier.qualify(x)
        print(qualification)
    cv.imshow('Video', frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break


I0000 00:00:1701722257.231426       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.327831       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.394188       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine


[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=540, y=394), end=Point(x=795, y=649)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=537, y=392), end=Point(x=793, y=648)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=539, y=394), end=Point(x=794, y=649)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=538, y=400), end=Point(x=788, y=650)))]


I0000 00:00:1701722257.461165       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.527534       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.595368       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.662799       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine


[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=536, y=393), end=Point(x=792, y=649)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=532, y=394), end=Point(x=785, y=647)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=529, y=395), end=Point(x=791, y=657)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=538, y=409), end=Point(x=786, y=657)))]


I0000 00:00:1701722257.729295       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.794401       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.862708       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722257.929147       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine


...

[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=553, y=406), end=Point(x=775, y=628)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=553, y=410), end=Point(x=758, y=615)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=563, y=400), end=Point(x=751, y=588)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=535, y=402), end=Point(x=754, y=621)))]


I0000 00:00:1701722259.593051       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722259.660496       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722259.727929       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722259.793512       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine


[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=524, y=399), end=Point(x=757, y=632)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=511, y=403), end=Point(x=749, y=641)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=519, y=404), end=Point(x=750, y=635)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=503, y=383), end=Point(x=760, y=640)))]
[Person(age='42', emotions=[Emotions(name='happy', quantity=0.42)], bounding_box=BoundingBox(origin=Point(x=502, y=403), end=Point(x=744, y=645)))]


I0000 00:00:1701722259.859155       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine
I0000 00:00:1701722259.926226       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.12.7), renderer: AMD Radeon Pro 5300M OpenGL Engine


KeyboardInterrupt: 